# Unit 1 Assignment

In this assignment, we will focus on sports analytics. This data set is made available by http://www.baseball-reference.com. It contains data about professional baseball (MLB) games played in the 2016 season. There are 2,427 games in the data set. Each row represents a single game. The goal is to predict the attendance at a home team’s game. This is an important task because most franchises want to predict the number of attendees for a variety of reasons including profits.

## Description of Variables

The description of variables are provided in "Baseball - Data Dictionary.docx"

## Goal

Use the **baseball.csv** data set and build a model to predict **attendance_binary**.

## Submission:

Please save and submit this Jupyter notebook file. The correctness of the code matters for your grade. **Readability and organization of your code is also important.** You may lose points for submitting unreadable/undecipherable code. Therefore, use markdown cells to create sections, and use comments where necessary.


# Section 1: (6 points in total)

## Data Prep (5.5 points)

In [1]:
# Common imports
import numpy as np
import pandas as pd
np.random.seed(50)

In [2]:
mlb = pd.read_csv("baseball.csv")
mlb.head()

,attendance_binary,previous_attendance,previous_away_team_errors,previous_away_team_hits,previous_away_team_runs,game_type,previous_game_type,previous_home_team_errors,previous_home_team_hits,previous_home_team_runs,game_day,previous_game_day,temperature,wind_speed,sky,previous_game_duration,previous_homewin
0,0,43683,2,6,2,Night Game,Day Game,0,6,6,Wednesday,Monday,55,24,Overcast,2.933333,1
1,0,45785,0,7,2,Night Game,Day Game,0,10,3,Wednesday,Monday,48,7,Unknown,2.800000,1
2,0,48282,0,8,4,Night Game,Day Game,2,4,3,Wednesday,Monday,65,10,Cloudy,3.383333,0
3,0,21830,0,9,6,Day Game,Night Game,0,15,11,Wednesday,Tuesday,77,0,In Dome,3.233333,1
4,0,49289,2,4,2,Night Game,Day Game,1,1,3,Tuesday,Monday,81,12,Cloudy,2.633333,1


In [3]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(mlb, test_size=0.3)

In [4]:
train.isna().sum(), test.isna().sum()

(attendance_binary            0
 previous_attendance          0
 previous_away_team_errors    0
 previous_away_team_hits      0
 previous_away_team_runs      0
 game_type                    0
 previous_game_type           0
 previous_home_team_errors    0
 previous_home_team_hits      0
 previous_home_team_runs      0
 game_day                     0
 previous_game_day            0
 temperature                  0
 wind_speed                   0
 sky                          0
 previous_game_duration       0
 previous_homewin             0
 dtype: int64,
 attendance_binary            0
 previous_attendance          0
 previous_away_team_errors    0
 previous_away_team_hits      0
 previous_away_team_runs      0
 game_type                    0
 previous_game_type           0
 previous_home_team_errors    0
 previous_home_team_hits      0
 previous_home_team_runs      0
 game_day                     0
 previous_game_day            0
 temperature                  0
 wind_speed              

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [6]:
train_y = train[['attendance_binary']]
test_y = test[['attendance_binary']]

train_inputs = train.drop(['attendance_binary'], axis=1)
test_inputs = test.drop(['attendance_binary'], axis=1)

In [7]:
train_inputs.dtypes

previous_attendance            int64
previous_away_team_errors      int64
previous_away_team_hits        int64
previous_away_team_runs        int64
game_type                     object
previous_game_type            object
previous_home_team_errors      int64
previous_home_team_hits        int64
previous_home_team_runs        int64
game_day                      object
previous_game_day             object
temperature                    int64
wind_speed                     int64
sky                           object
previous_game_duration       float64
previous_homewin               int64
dtype: object

In [8]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

# Identify the binary columns
binary_columns = ['previous_homewin']

In [9]:
for col in binary_columns:
    numeric_columns.remove(col)

In [10]:
binary_columns

['previous_homewin']

In [11]:
numeric_columns

['previous_attendance',
 'previous_away_team_errors',
 'previous_away_team_hits',
 'previous_away_team_runs',
 'previous_home_team_errors',
 'previous_home_team_hits',
 'previous_home_team_runs',
 'temperature',
 'wind_speed',
 'previous_game_duration']

In [12]:
categorical_columns

['game_type', 'previous_game_type', 'game_day', 'previous_game_day', 'sky']

In [13]:
# There are no missing value so this is probably not needed

numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [14]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [15]:
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [16]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='passthrough')

In [17]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)
train_x

array([[-0.0298191 , -0.72817732,  0.64354552, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.61619157,  1.78237109,  1.50623522, ...,  1.        ,
         0.        ,  0.        ],
       [-0.62510121,  0.52709688, -0.21914418, ...,  0.        ,
         1.        ,  1.        ],
       ...,
       [-0.73630224, -0.72817732, -0.50670742, ...,  1.        ,
         0.        ,  1.        ],
       [ 1.0285495 , -0.72817732,  0.93110875, ...,  1.        ,
         0.        ,  0.        ],
       [-1.02470165,  0.52709688,  0.06841905, ...,  0.        ,
         0.        ,  1.        ]])

In [18]:
train_x.shape

(1698, 37)

In [19]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

array([[ 0.36194604, -0.72817732, -0.50670742, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.69736291, -0.72817732, -0.79427065, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.66310857,  1.78237109, -0.79427065, ...,  1.        ,
         0.        ,  1.        ],
       ...,
       [-1.11500854, -0.72817732,  1.50623522, ...,  0.        ,
         0.        ,  0.        ],
       [-0.61726591,  0.52709688,  0.64354552, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.53151003, -0.72817732, -0.79427065, ...,  1.        ,
         0.        ,  1.        ]])

In [20]:
test_x.shape

(729, 37)

## Find the Baseline (0.5 point)

In [21]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(train_x, train_y)

DummyClassifier(strategy='most_frequent')

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
# # This is the baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)
baseline_train_acc = accuracy_score(train_y, dummy_train_pred)
print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.5217903415783275


In [24]:
# This is the baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)
baseline_test_acc = accuracy_score(test_y, dummy_test_pred)
print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.5116598079561042


# Section 2: (3 points in total)

Build three different SVM models (by changing the kernels, regularization, etc.). Generate their training and test values. Each model is worth 1 point. 

(Add cells as needed)

## SVM Model 1:

In [25]:
from sklearn.svm import SVC
lin_svm = SVC(kernel="linear")
lin_svm.fit(train_x, train_y)

C:\Users\Sophie\anaconda3\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='linear')

In [26]:
from sklearn.metrics import accuracy_score

In [27]:
#Predict the train values
train_y_pred = lin_svm.predict(train_x)
#Train accuracy
accuracy_score(train_y, train_y_pred)

0.8392226148409894

In [28]:
#Predict the test values
test_y_pred = lin_svm.predict(test_x)
#Test accuracy
accuracy_score(test_y, test_y_pred)

0.8189300411522634

In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, test_y_pred)

array([[290,  66],
       [ 66, 307]], dtype=int64)

## SVM Model 2:

In [30]:
from sklearn.svm import SVC
pol_svm = SVC(kernel="poly", degree=3, coef0=1, C=.005)
pol_svm.fit(train_x, train_y)

C:\Users\Sophie\anaconda3\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=0.005, coef0=1, kernel='poly')

In [31]:
#Predict the train values
train_y_pred = pol_svm.predict(train_x)
#Train accuracy
accuracy_score(train_y, train_y_pred)

0.8368669022379269

In [32]:
#Predict the test values
test_y_pred = pol_svm.predict(test_x)
#Test accuracy
accuracy_score(test_y, test_y_pred)

0.7969821673525377

In [33]:
confusion_matrix(test_y, test_y_pred)

array([[264,  92],
       [ 56, 317]], dtype=int64)

## SVM Model 3:

In [34]:
rbf_svm = SVC(kernel="rbf", C=0.01, gamma='scale')
rbf_svm.fit(train_x, train_y)

C:\Users\Sophie\anaconda3\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=0.01)

In [35]:
#Predict the train values
train_y_pred = rbf_svm.predict(train_x)
#Train accuracy
accuracy_score(train_y, train_y_pred)

0.5547703180212014

In [36]:
#Predict the test values
test_y_pred = rbf_svm.predict(test_x)
#Test accuracy
accuracy_score(test_y, test_y_pred)

0.5349794238683128

In [37]:
confusion_matrix(test_y, test_y_pred)

array([[ 18, 338],
       [  1, 372]], dtype=int64)

# Section 3: (3 points in total)

Build two different SGD models (by changing the penalty, etc. or adding polynomial terms) and one LogisticRregression model. Generate their training and test values. Each model is worth 1 point.

(Add cells as needed)

## SGD Model 1:

In [38]:
from sklearn.linear_model import SGDClassifier 

sgd_logreg = SGDClassifier(max_iter=100, penalty=None, eta0=0.01) 

sgd_logreg.fit(train_x, train_y)

C:\Users\Sophie\anaconda3\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(eta0=0.01, max_iter=100, penalty=None)

In [39]:
#Predict the train values
train_y_pred = sgd_logreg.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.8168433451118964

In [40]:
#Predict the test values
test_y_pred = sgd_logreg.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.7873799725651578

In [41]:
confusion_matrix(test_y, test_y_pred)

array([[248, 108],
       [ 47, 326]], dtype=int64)

## SGD Model 2:

In [42]:
from sklearn.linear_model import SGDClassifier 

sgd_logreg = SGDClassifier(max_iter=100, penalty='elasticnet', l1_ratio=0.5, eta0=0.01) 

sgd_logreg.fit(train_x, train_y)

C:\Users\Sophie\anaconda3\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(eta0=0.01, l1_ratio=0.5, max_iter=100, penalty='elasticnet')

In [43]:
#Predict the train values
train_y_pred = sgd_logreg.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.8227326266195524

In [44]:
#Predict the test values
test_y_pred = sgd_logreg.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.7791495198902606

In [45]:
confusion_matrix(test_y, test_y_pred)

array([[227, 129],
       [ 32, 341]], dtype=int64)

## LogisticRegression Model:

In [46]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=500,penalty='none')
log_reg.fit(train_x, train_y)

C:\Users\Sophie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\Sophie\anaconda3\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=500, penalty='none')

In [47]:
log_reg.predict(test_x)

array([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,

In [48]:
predictions = pd.DataFrame(log_reg.predict(test_x), columns=['Predicted'])
predictions

,Predicted
0,1
1,1
2,1
3,1
4,0
...,...
724,0
725,1
726,0
727,0


In [49]:
predictions['Actual'] = np.array(test_y)
predictions

,Predicted,Actual
0,1,1
1,1,1
2,1,1
3,1,1
4,0,0
...,...,...
724,0,0
725,1,0
726,0,0
727,0,0


In [50]:
#Predict the train values
train_y_pred = log_reg.predict(train_x)
#Train accuracy
accuracy_score(train_y, train_y_pred)

0.842756183745583

In [51]:
#Predict the test values
test_y_pred = log_reg.predict(test_x)
#Test accuracy
accuracy_score(test_y, test_y_pred)

0.8161865569272977

In [52]:
confusion_matrix(test_y, test_y_pred)

array([[290,  66],
       [ 68, 305]], dtype=int64)

# Discussion (3 points in total)


## List the train and test values of each model you built (1 point)

## Which model performs the best and why? (0.5 points) How does it compare to baseline? (0.5 points)

Hint: The best model is the one that has the highest TEST score (regardless of any of the training values). If you select your model based on TRAIN values, you will lose points.

## Is there any evidence of overfitting in the best model, why or why not? If there is, what did you do about it? (0.5 points)

## Is there any evidence of overfitting in the other models (besides the best model), why or why not? If there is, what did you do about it? (0.5 points)